In [2]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
%matplotlib inline
import time
from tqdm import tqdm
import multiprocessing as mp
import pandas as pd

matrix_interact = np.loadtxt('C:\\Users\\bryve\\OneDrive\\psychiatry\\data_new\\nodes_interaction.csv', delimiter=';')
cal_type = np.ones(9)
cal_type[3] = 2

In [3]:
def Boolean_f(interact_net,cal_type,input_state):
    ##cal_type为1时为并运算，为2时为合运算
    nodes_num=len(cal_type)
    S_temp=np.zeros(nodes_num)
    for i in range(3):
        if cal_type[i]>0:
            S_temp[i]=input_state[i]
    for i in range(nodes_num):
        for j in range(nodes_num):
            S_temp[i] +=interact_net[j,i]*input_state[j]
    for i in range(nodes_num):
        if cal_type[i]==1:
            if S_temp[i]>1:
                S_temp[i]=1
            elif S_temp[i]<0:
                S_temp[i]=0
        elif cal_type[i]==2:
            if S_temp[i]<2:
                S_temp[i]=0
            elif S_temp[i]==2:
                S_temp[i]==1
        elif cal_type[i]==0:
            S_temp[i]=0
    return S_temp

In [4]:
###给定函数的2次数，输出二进制状态函数
def Init_boolState(st_id,nodes_num):
    bin_obj = bin(int(st_id))[2:]
    bin_str = bin_obj.rjust(nodes_num,'0')
    o_arr = []
    for ii in range(len(bin_str)):
        o_arr.append(int(bin_str[len(bin_str)-ii-1]))
    return o_arr    

In [5]:
def Stable_f(ini_state,interact_net,cal_type):
    S1=Boolean_f(interact_net,cal_type,ini_state)
    S2=Boolean_f(interact_net,cal_type,S1)
    if (S2==S1).all():
        return S2
    else:
        t=0
        while (S2!=S1).any() and t<100:
            S1=Boolean_f(interact_net,cal_type,S2)
            S2=Boolean_f(interact_net,cal_type,S1)
            t+=1
        return S2

In [6]:
def State2num(input_state,nodes_num):
    num=0
    for a in range(nodes_num):
        num+=input_state[a]*(2**a)
    return(num)

In [7]:
def Booleanf_fix(interact_net,cal_type,input_state):
    ##cal_type为1时为交运算，为2时为并运算
    nodes_num=len(cal_type)
    S_temp=np.zeros(nodes_num)
    for i in range(3):
        if cal_type[i]>0:
            S_temp[i]=input_state[i]
    for i in range(nodes_num):
        for j in range(nodes_num):
            S_temp[i] +=interact_net[j,i]*input_state[j]
    for i in range(nodes_num):
        if cal_type[i]==1:
            if S_temp[i]>1:
                S_temp[i]=1
            elif S_temp[i]<0:
                S_temp[i]=0
        elif cal_type[i]==2:
            if S_temp[i]<2:
                S_temp[i]=0
            elif S_temp[i]==2:
                S_temp[i]==1
        elif cal_type[i]==0:
            S_temp[i]=0
    return S_temp

Standard Boolean Networks

In [12]:
###标准主函数
###标准布尔网络中的初始状态为0状态、mGluR激活、NMDAR激活、PKA激活

##0状态的吸引子子集
state_trans0=np.zeros([64,512])
nodes_si=np.zeros(9)
for i in range(64):
    nodes_si[3:9]=Init_boolState(i,6)
    nodes_ss=Stable_f(nodes_si,matrix_interact,cal_type)
    stable_id=int(State2num(nodes_ss,9))
    state_trans0[i,stable_id]=1
data0 = pd.DataFrame(state_trans0)
data0.to_csv('n_state_0.csv')

##mGluR输入的吸引子子集
state_transm=np.zeros([64,512])
nodes_sim=np.zeros(9)
nodes_sim[0]=1
for i in range(64):
    nodes_sim[3:9]=Init_boolState(i,6)
    nodes_ssm=Stable_f(nodes_sim,matrix_interact,cal_type)
    stable_id=int(State2num(nodes_ssm,9))
    state_transm[i,stable_id]=1
dataM = pd.DataFrame(state_transm)
dataM.to_csv('n_state_m.csv')

##NMDAR输入的吸引子子集
state_transN=np.zeros([128,512])
nodes_siN=np.zeros(9)
nodes_siN[1]=1
for i in range(128):
    nodes_siN[0]=Init_boolState(i,7)[0]
    nodes_siN[3:9]=Init_boolState(i,7)[1:7]
    nodes_ssN=Stable_f(nodes_siN,matrix_interact,cal_type)
    stable_id=int(State2num(nodes_ssN,9))
    state_transN[i,stable_id]=1
dataN = pd.DataFrame(state_transN)
dataN.to_csv('n_state_N.csv')

##PKA输入的吸引子子集
state_transA=np.zeros([256,512])
nodes_siA=np.zeros(9)
nodes_siA[2]=1
for i in range(256):
    nodes_siA[0:2]=Init_boolState(i,8)[0:2]
    nodes_siA[3:9]=Init_boolState(i,8)[2:8]
    nodes_ssA=Stable_f(nodes_siA,matrix_interact,cal_type)
    stable_id=int(State2num(nodes_ssA,9))
    state_transA[i,stable_id]=1
dataA = pd.DataFrame(state_transA)
dataA.to_csv('n_state_A.csv')

In [15]:
##敲除节点的动力学演化

##类型1删除PKC及其他
matrix_koC = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koC.csv', delimiter=';')
state_transC=np.zeros([256,256])
cal_type_C = np.ones(8)
nodes_siC=np.zeros(8)
for i in range(256):
    nodes_siC=Init_boolState(i,8)
    nodes_ssC=Stable_f(nodes_siC,matrix_koC,cal_type_C)
    stable_id=int(State2num(nodes_ssC,8))
    state_transC[i,stable_id]=1
dataC = pd.DataFrame(state_transC)
dataC.to_csv('n_state_koC.csv')

##类型2删除IP3
matrix_koI3 = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koIP3.csv', delimiter=';')
state_transI3=np.zeros([256,256])
cal_type_I3 = np.ones(8)
nodes_siI3=np.zeros(8)

for i in range(256):
    nodes_siI3=Init_boolState(i,8)
    nodes_ssI3=Stable_f(nodes_siI3,matrix_koI3,cal_type_I3)
    stable_id=int(State2num(nodes_ssI3,8))
    state_transI3[i,stable_id]=1
dataI3 = pd.DataFrame(state_transI3)
dataI3.to_csv('n_state_koI3.csv')

##类型3删除NSF、SAP97
matrix_koNSF = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koNSF.csv', delimiter=';')
state_transNSF=np.zeros([512,512])
cal_type_NSF = np.ones(9)
nodes_siNSF=np.zeros(9)
for i in range(512):
    nodes_siNSF=Init_boolState(i,9)
    nodes_ssNSF=Stable_f(nodes_siNSF,matrix_koNSF,cal_type_NSF)
    stable_id=int(State2num(nodes_ssNSF,9))
    state_transNSF[i,stable_id]=1
dataNSF = pd.DataFrame(state_transNSF)
dataNSF.to_csv('n_state_koNSF.csv')

##类型4删除PSD95orTARP
matrix_koPSD = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koPSD.csv', delimiter=';')
state_transPSD=np.zeros([256,256])
cal_type_PSD = np.ones(8)
nodes_siPSD=np.zeros(8)

for i in range(256):
    nodes_siPSD=Init_boolState(i,8)
    nodes_ssPSD=Stable_f(nodes_siPSD,matrix_koPSD,cal_type_PSD)
    stable_id=int(State2num(nodes_ssPSD,8))
    state_transPSD[i,stable_id]=1
dataPSD = pd.DataFrame(state_transPSD)
dataPSD.to_csv('n_state_koPSD.csv')

##类型5删除CaM
matrix_koCM = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koCM.csv', delimiter=';')
state_transCM=np.zeros([128,128])
cal_type_CM = np.ones(7)
nodes_siCM=np.zeros(7)

for i in range(128):
    nodes_siCM=Init_boolState(i,7)
    nodes_ssCM=Stable_f(nodes_siCM,matrix_koCM,cal_type_CM)
    stable_id=int(State2num(nodes_ssCM,7))
    state_transCM[i,stable_id]=1
dataCM = pd.DataFrame(state_transCM)
dataCM.to_csv('n_state_koCM.csv')

##类型6删除CaN
matrix_koCN = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koCN.csv', delimiter=';')
state_transCN=np.zeros([128,128])
cal_type_CN = np.ones(7)
nodes_siCN=np.zeros(7)

for i in range(128):
    nodes_siCN=Init_boolState(i,7)
    nodes_ssCN=Stable_f(nodes_siCN,matrix_koCN,cal_type_CN)
    stable_id=int(State2num(nodes_ssCN,7))
    state_transCN[i,stable_id]=1
dataCN = pd.DataFrame(state_transCN)
dataCN.to_csv('n_state_koCN.csv')

##类型7删除CaMKII
matrix_koCMK = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koCMK.csv', delimiter=';')
state_transCMK=np.zeros([256,256])
cal_type_CMK=np.ones(8)
nodes_siCMK=np.zeros(8)

for i in range(256):
    nodes_siCMK=Init_boolState(i,8)
    nodes_ssCMK=Stable_f(nodes_siCMK,matrix_koCMK,cal_type_CMK)
    stable_id=int(State2num(nodes_ssCMK,8))
    state_transCMK[i,stable_id]=1
dataCMK = pd.DataFrame(state_transCMK)
dataCMK.to_csv('n_state_koCMK.csv')



In [33]:
##类型8删除PP1
matrix_koP1 = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koP1.csv', delimiter=';')
state_transP1=np.zeros([512,512])
cal_type_P1 = np.ones(9)
cal_type_P1[3]=2
nodes_siP1=np.zeros(9)

for i in range(512):
    nodes_siP1=Init_boolState(i,9)
    nodes_ssP1=Stable_f(nodes_siP1,matrix_koP1,cal_type_P1)
    stable_id=int(State2num(nodes_ssP1,9))
    state_transP1[i,stable_id]=1
dataP1 = pd.DataFrame(state_transP1)
dataP1.to_csv('n_state_koP1.csv')


##类型9删除GRIP
matrix_koGRIP = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_koGRIP.csv', delimiter=';')
state_transGRIP=np.zeros([512,512])
cal_type_GRIP = np.ones(9)
nodes_siGRIP=np.zeros(9)
for i in range(512):
    nodes_siGRIP=Init_boolState(i,9)
    nodes_ssGRIP=Stable_f(nodes_siGRIP,matrix_koGRIP,cal_type_GRIP)
    stable_id=int(State2num(nodes_ssGRIP,9))
    state_transGRIP[i,stable_id]=1
dataGRIP = pd.DataFrame(state_transGRIP)
dataGRIP.to_csv('n_state_koGRIP.csv')

In [29]:
##固定节点高表达

##TARP or PSD95 overexpression
matrix_oePSD = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_oePSD.csv', delimiter=';')
state_transPSD=np.zeros([512,512])
cal_type_PSD = np.ones(9)
nodes_siPSD=np.zeros(9)
cal_type_PSD[3] = 2
nodes_siPSD[7]=1
for i in range(512):
    nodes_siPSD=Init_boolState(i,9)
    nodes_ssPSD=Stable_f(nodes_siPSD,matrix_oePSD,cal_type_PSD)
    stable_id=int(State2num(nodes_ssPSD,9))
    state_transPSD[i,stable_id]=1
dataTAR = pd.DataFrame(state_transPSD)
dataTAR.to_csv('n_state_oePSD.csv')

##CaMKII overexpression
matrix_oeCMK = np.loadtxt('C:\\Users\\qysys\\OneDrive\\psychiatry\\data\\nodes_intc_oeCMK.csv', delimiter=';')
state_transCMK=np.zeros([512,512])
cal_type_CMK = np.ones(9)
nodes_siCMK=np.zeros(9)
cal_type_CMK[3] = 2
nodes_siCMK[6]=1
for i in range(512):
    nodes_siCMK=Init_boolState(i,9)
    nodes_ssCMK=Stable_f(nodes_siCMK,matrix_oeCMK,cal_type_CMK)
    stable_id=int(State2num(nodes_ssCMK,9))
    state_transCMK[i,stable_id]=1
dataCMK = pd.DataFrame(state_transCMK)
dataCMK.to_csv('n_state_oeCMK.csv')